In [ ]:
path = "c:/Users/Nathalie/Nextcloud/LADi/Orthografie Trainer/Code/04_bias_mitigation/"

import sys

sys.path.insert(1, path + "classes")

from DecisionTree_insession import in_session_decision_tree
from KNN_insession import in_session_KNN
from MLP_insession import in_session_MLP
from MetricsClass import Evaluation
from Plotting import Plots

import pandas as pd
import seaborn as sns
import pickle

Aggregation Bias Mitigation

Group 1: Book 0

In [ ]:
## define feature cols
feature_cols = [
    "Erstloesung",
    "Schussel",
    "Erfolg",
    "Schwierigkeit",
    "ist_Schulzeit",
    "MehrfachFalsch",
    "vorher_abgebrochen",
    "Fehler",
    "Klassenstufe",
    "Jahredabei",
    "Testposition__pruefung",
    "Testposition__training",
    "Testposition__version",
    "Art__GK",
    "Art__GR",
    "Art__GZ",
    "Art__K",
    "Art__LB",
    "UserAttribut",
    "OrderNumber",
    "steps"
]

Decision Tree Classifier

In [ ]:
DTE = in_session_decision_tree()
DTE.set_range(2, 61)
DTE.set_majority_group("buecher_0")
DTE.set_demographic_category("Buecher")

DTE.load_matrices_aggregation_bias("05_aggregation_bias_mitigation/books/matrix_buch0", ".pkl", 0)
DTE.set_feature_cols(feature_cols)

metrics_DTE_buch0 = DTE.loop_matrices("books/matrix_buch0", 5, 1, 2)

KNN

In [ ]:
KNN = in_session_KNN()
KNN.set_range(2, 61)
KNN.set_majority_group("buecher_0")
KNN.set_demographic_category("Buecher")

KNN.load_matrices_aggregation_bias("05_aggregation_bias_mitigation/books/matrix_buch0", ".pkl", 0)

KNN.set_feature_cols(feature_cols)

KNN.set_majority_group("buecher_0")
metrics_KNN_buch0 = KNN.loop_matrices("books/matrix_buch0", 2, "uniform")

MLP

In [ ]:
MLP = in_session_MLP()
MLP.set_range(2, 61)

MLP.set_majority_group("buecher_0")
MLP.set_demographic_category("Buecher")

MLP.load_matrices_aggregation_bias("05_aggregation_bias_mitigation/books/matrix_buch0", ".pkl", 0)

MLP.set_feature_cols(feature_cols)

metrics_MLP_buch0 = MLP.loop_matrices(
    "books/matrix_buch0", "Adam", "binary_crossentropy", "accuracy", 21, 21
)

Group 2: Buch 1

Decision Tree Classifier

In [ ]:
DTE = in_session_decision_tree()
DTE.set_range(2, 61)
DTE.set_majority_group("buecher_1")
DTE.set_demographic_category("Buecher")

DTE.load_matrices_aggregation_bias("05_aggregation_bias_mitigation/books/matrix_buch1", ".pkl", 1)

DTE.set_feature_cols(feature_cols)


metrics_DTE_buch1 = DTE.loop_matrices("books/matrix_buch1", 5, 1, 2)

KNN

In [ ]:
KNN = in_session_KNN()
KNN.set_range(2, 61)
KNN.set_majority_group("buecher_1")
KNN.set_demographic_category("Buecher")

KNN.load_matrices_aggregation_bias("05_aggregation_bias_mitigation/books/matrix_buch1", ".pkl", 1)

KNN.set_feature_cols(feature_cols)

metrics_KNN_buch1 = KNN.loop_matrices("books/matrix_buch1", 2, "uniform")

MLP

In [ ]:
MLP = in_session_MLP()
MLP.set_range(2, 61)

MLP.set_majority_group("buecher_1")
MLP.set_demographic_category("Buecher")

MLP.load_matrices_aggregation_bias("05_aggregation_bias_mitigation/books/matrix_buch1", ".pkl", 1)

MLP.set_feature_cols(feature_cols)

metrics_MLP_buch1 = MLP.loop_matrices(
    "books/matrix_buch1", "Adam", "binary_crossentropy", "accuracy", 21, 21
)

Model metrics

In [ ]:
metrics = pd.concat([metrics_DTE_buch1, metrics_DTE_buch0, metrics_KNN_buch1, metrics_KNN_buch0, metrics_MLP_buch1, metrics_MLP_buch0])


In [ ]:
## construct dfs for both groups from metric df
grouped = metrics.groupby(metrics.subgroup)
df_buch0 = grouped.get_group("buecher_0")
df_buch1 = grouped.get_group("buecher_1")

Evaluation

In [ ]:
# evaluate metrics for fairness
evaluation_obj = Evaluation(metrics)
evaluation_obj.set_demographic_category("Buecher")
evaluation_obj.set_minority_group("buecher_1")
evaluation_obj.set_majority_group("buecher_0")
df_buecher = evaluation_obj.evaluate()

In [ ]:
# show table
table = evaluation_obj.showTable(df_buecher)
table

In [ ]:
# save
writer = pd.ExcelWriter("df_HLE_AggBias.xlsx", engine="xlsxwriter")
df_buecher.to_excel(writer, sheet_name="HLE")
writer.save()